In [2]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
import imageio
# 创建一个Tkinter窗口
root = tk.Tk()
 
# 弹出文件选择对话框
file_path = filedialog.askopenfilename() 
print("选择的文件路径:", file_path)

root.destroy()
def is_gif_file(file_path):
    with open(file_path, 'rb') as f:
        return f.read(6) == b'GIF89a' or f.read(6) == b'GIF87a'
if is_gif_file(file_path): 
    gif_image = imageio.imread(file_path)
    img= cv2.cvtColor(gif_image, cv2.COLOR_RGB2BGR)
else:
    img=cv2.imread(file_path)
img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


选择的文件路径: C:/Users/Administrator/Pictures/wishes.jpg


In [3]:

# 使用Canny算法检测边缘
edges = cv2.Canny(gray, 50, 150)

# 查找轮廓
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 对每个轮廓进行膨胀操作
kernel = np.ones((5,5),int)

#contours = [cv2.dilate(cnt, kernel, iterations = 1) for cnt in contours]

# 遍历每个轮廓
for i, contour in enumerate(contours):
    # 计算轮廓的边界框
    x, y, w, h = cv2.boundingRect(contour)

    # 在原图上画出边界框
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 1)

    # 提取边界框内的部分，保存为新的图片
    roi = img[y:y+h, x:x+w]
    cv2.imwrite('data/output/object_{}_{}.jpg'.format(i,np.random.randint(1000,9999)), roi)

# 显示带有边界框的图像
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
# 使用二值化处理
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# 使用形态学膨胀操作，连接断开的笔画
kernel = np.ones((5,5),np.uint8)
dilated = cv2.dilate(binary, kernel, iterations = 1)

# 使用闭运算填充断开的笔画
closed = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)

# 查找轮廓
contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 遍历每个轮廓
for i, contour in enumerate(contours):
    # 计算轮廓的边界框
    x, y, w, h = cv2.boundingRect(contour)

    # 在原图上画出边界框
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0),1)

    # 提取边界框内的部分，保存为新的图片
    roi = img[y:y+h, x:x+w]
    cv2.imwrite('data/output/object_{}_{}.jpg'.format(i,np.random.randint(1000,9999)),roi)

# 显示带有边界框的图像
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
root = tk.Tk()
 
# 弹出文件选择对话框
file_path = filedialog.askopenfilename() 
print("选择的文件路径:", file_path)

root.destroy()
def is_gif_file(file_path):
    return file_path.split('.')[-1]=='gif' or file_path.split('.')[-1]=='GIF'
if is_gif_file(file_path): 
    gif_image = imageio.imread(file_path)
    img= cv2.cvtColor(gif_image, cv2.COLOR_RGB2BGR)
else:
    img=cv2.imread(file_path)
img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

image=img
clone = image.copy()

# 创建一个窗口来显示图片
cv2.namedWindow('image')

# 初始化变量
drawing = False
points = []
counter = 1
contours = []
# 定义鼠标事件的回调函数
def draw_contour(event, x, y, flags, param):
    global drawing, points, clone, counter

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        points = [(x, y)]

    if event == cv2.EVENT_LBUTTONUP:
        drawing = False
        points.append((x, y))
        cv2.rectangle(clone, points[0], points[1], (0, 255, 0), 2)
        cv2.imshow('image', clone)

        # 提取标出的轮廓区域
        roi = image[points[0][1]:points[1][1], points[0][0]:points[1][0]]
        # 保存为单独的图片文件
        filename = f'data/output/word_{counter}_{str(np.random.randint(10000,99999))}.jpg'
        
        cv2.imwrite(filename, roi)
        print(f'保存成功！文件名：{filename}')
        counter += 1
        # 添加轮廓到列表中
        contours.append(points)  
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            clone = image.copy()
            cv2.rectangle(clone, points[0], (x, y), (0, 255, 0), 1)
            cv2.imshow('image', clone)
  
# 注册鼠标事件的回调函数
cv2.setMouseCallback('image', draw_contour)
# 绘制保存的轮廓
while True:
    cv2.imshow('image', clone)
    # & 0xFF用于把高于8位的值屏蔽掉
    key = cv2.waitKey(1) & 0xFF
    for contour in contours:
        cv2.rectangle(clone, contour[0], contour[1], (255, 0, 255), thickness=1)

    # 按下 'q' 键退出程序
    if key == ord('q'):
        break
        
# 关闭窗口
cv2.destroyAllWindows()

选择的文件路径: C:/Users/Administrator/Pictures/wishes.jpg
保存成功！文件名：data/output/word_1_65134.jpg
保存成功！文件名：data/output/word_2_72673.jpg
保存成功！文件名：data/output/word_3_85498.jpg
保存成功！文件名：data/output/word_4_75757.jpg
保存成功！文件名：data/output/word_5_37384.jpg
保存成功！文件名：data/output/word_6_30992.jpg
保存成功！文件名：data/output/word_7_40747.jpg
保存成功！文件名：data/output/word_8_62979.jpg
保存成功！文件名：data/output/word_9_91843.jpg
保存成功！文件名：data/output/word_10_87347.jpg
保存成功！文件名：data/output/word_11_93657.jpg
保存成功！文件名：data/output/word_12_65404.jpg
